In [168]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

In [169]:
df = pd.read_csv("../data/datos_donantes.csv")

In [170]:
df = df[df['donation_date'] < '2024-04-10']

In [171]:
# df['fecha'] = pd.to_datetime(df['donation_date'])

In [172]:
df = df.groupby('donation_date').sum()

In [173]:
fecha_inicio = df.index.min()
fecha_fin = df.index.max()
rango_fechas = pd.date_range(start=fecha_inicio, end=fecha_fin, freq='D')

In [174]:
# Crear un DataFrame con todas las fechas posibles
serie_temporal = pd.DataFrame(index=rango_fechas)

In [176]:
# Unir los datos de donaciones al DataFrame de fechas
serie_temporal = pd.merge(serie_temporal, df, left_index=True, right_index=True, how='left')

In [177]:
# Rellenar los valores faltantes con 0
serie_temporal['amount'].fillna(0, inplace=True)

In [178]:








serie_temporal

,name,email,date,is_partner,company,role_company,donation_frecuency,suscription_status,donation_type,amount,method_pay
2006-11-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
2006-11-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
2006-12-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
2006-12-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
2006-12-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
2024-04-03 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
2024-04-04 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN
2024-04-05 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN


In [167]:
serie_temporal.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6339 entries, 2006-11-29 00:00:00 to 2024-04-06 00:00:00
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                0 non-null      object 
 1   email               0 non-null      object 
 2   date                0 non-null      object 
 3   is_partner          0 non-null      float64
 4   company             0 non-null      object 
 5   role_company        0 non-null      object 
 6   donation_frecuency  0 non-null      float64
 7   suscription_status  0 non-null      float64
 8   donation_type       0 non-null      float64
 9   amount              6339 non-null   float64
 10  method_pay          0 non-null      float64
dtypes: float64(6), object(5)
memory usage: 594.3+ KB


In [185]:
df['fecha'] = pd.to_datetime(df.index)
df['año'] = df['fecha'].dt.year
df['mes'] = df['fecha'].dt.month
df['dia'] = df['fecha'].dt.day

In [146]:
df['año'].unique()

array([2021, 2022, 2014, 2018, 2024, 2012, 2020, 2016, 2007, 2017, 2019,
       2013, 2010, 2023, 2008, 2011, 2015, 2009, 2006])

In [181]:
df = df.drop(['name', 'email', 'date', 'is_partner', 'company', 'role_company', 'donation_frecuency', 'suscription_status', 'donation_type', 'method_pay'], axis=1)

In [187]:
df = df.drop('fecha', axis=1)

In [182]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2921 entries, 2006-11-29 to 2024-04-06
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   amount  2921 non-null   float64
dtypes: float64(1)
memory usage: 110.2+ KB


In [188]:
# Dividir los datos en características (X) y la variable objetivo (y)
X = df[['año', 'mes', 'dia']]
y = df['amount']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [189]:
train_mat = xgb.DMatrix(X_train)
test_mat = xgb.DMatrix(X_test)

In [190]:
# Definir el modelo
model = XGBRegressor()

In [191]:
# Entrenar el modelo
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [192]:
# Hacer predicciones en el conjunto de prueba
predictions = model.predict(X_test)

In [193]:
# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, predictions)
print("RMSE:", np.sqrt(mse))

RMSE: 2389.27606733232


In [194]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, predictions)
print("Coeficiente de determinación R^2:", r2)

Coeficiente de determinación R^2: 0.6208898101840967
